### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [809]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and
purchase_data = pd.read_csv(file_to_load)

# store into Pandas data frame
purchase_data_df = pd.DataFrame(purchase_data)

# read header and first five rows
purchase_data_df.head()


,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [810]:
# # Total number of player will be unique count of the SN

# total_players = len(purchase_data_df["SN"].unique())
# total_players

# # Place all of the data found into a summary DataFrame
# total_players_df = pd.DataFrame({"Total Players": [total_players]
#                               })
# total_players_df


In [811]:
# Total number of player will be unique count of the SN

total_players = len(purchase_data_df["SN"].unique())
total_players

# Place all of the data found into a summary DataFrame
total_players_df = pd.DataFrame({"Total Players": [total_players]
                              })
total_players_df

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [812]:
# This section will create variables to hold the Purchasing Analysis values to be creaed in a new 

# first will capture the unique count of 

unique_items = len(purchase_data_df["Item ID"].unique())
unique_items

# second create the average price
average_price = round((purchase_data_df["Price"]).mean(),2)
average_price

# third create the number of purchases
purchase_count = len(purchase_data_df["Purchase ID"])
purchase_count

# fourth create the total revenue

total_revenue = purchase_data_df["Price"].sum()
total_revenue

# this cell will now create the "Purchaseing Analysis (Total) dataframe"

# Create a DataFrame of paintings using a list of dictionaries
purchasing_analysis_df = pd.DataFrame([
    {"Number of Unique Items": unique_items,
     "Average Price": average_price,
    "Number of Purchases": purchase_count,
    "Total Revenue": total_revenue}
])
purchasing_analysis_df["Average Price"] = purchasing_analysis_df["Average Price"].map("${:.2f}".format)
purchasing_analysis_df["Total Revenue"] = purchasing_analysis_df["Total Revenue"].map("${:,.2f}".format)
purchasing_analysis_df



,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [813]:
# Use existing variable for player count to create addtinoal variable to hold the gender counts and
# percentages;

# Note deduplication appears to be needed based on "SN"
# Create a df to capture just "SN" and "Gender"
SN_dedup_df = pd.DataFrame(purchase_data_df[["SN","Gender"]])


# create df to capture unique list of players and their genger
unique_client_df = pd.DataFrame(SN_dedup_df.drop_duplicates(subset=["SN"], keep='first', 
inplace=False, ignore_index=False))
unique_client_df = unique_client_df.reset_index(drop=True)
unique_client_df
gend_dem_df = pd.DataFrame(unique_client_df["Gender"].value_counts())
gend_dem_df = gend_dem_df.reset_index(drop=False)
gend_dem_df.head()

# calculatin the percentage column

gend_dem_df['Percentage of Players'] = round((gend_dem_df['Gender'] / 
                  gend_dem_df['Gender'].sum()) * 100, 2)
gend_dem_df["Percentage of Players"] = gend_dem_df["Percentage of Players"].map("{:.2f}%".format)
gend_dem_df

gend_dem_df.columns = ["Gender", "Total Count", "Percentage of Players"]
gend_dem_df




,Gender,Total Count,Percentage of Players
0,Male,484,84.03%
1,Female,81,14.06%
2,Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [814]:
# purchasing analysis by gender can be created by using a groupby function leveraging gender
# Using GroupBy in order to separate the data into fields according to "gender" values
grouped_purchase_data_df  = purchase_data_df.groupby(['Gender']).sum()
grouped_purchase_data_df

# data frame to hold the total purchase value grouped by gender
pa_tpv = pd.DataFrame(grouped_purchase_data_df["Price"])
pa_tpv = pa_tpv.rename(
    columns={"Price": "Total Purchase Value"})
pa_tpv



,Total Purchase Value
Gender,
Female,361.94
Male,1967.64
Other / Non-Disclosed,50.19


In [815]:
# Creating Dataframe to hold purchase count by gender
grouped_purchase_data_df  = purchase_data_df.groupby(['Gender']).count()
pa_pc = pd.DataFrame(grouped_purchase_data_df["Purchase ID"])
pa_pc = pa_pc.rename(
    columns={"Purchase ID": "Purchase Count"})
pa_pc

,Purchase Count
Gender,
Female,113
Male,652
Other / Non-Disclosed,15


In [816]:
# merging the purchase count and total purchase value
merged_pa = pd.merge(pa_pc, pa_tpv, on="Gender", how="right")
merged_pa


# calculated the average purchase price, add to the data frame and map the 
# total purchase value and total purchase prices
average_puchase_price = round(merged_pa.iloc[:,1] / merged_pa.iloc[:,0],2)
merged_pa["Average Purchase Price"] = average_puchase_price
merged_pa


,Purchase Count,Total Purchase Value,Average Purchase Price
Gender,,,
Female,113,361.94,3.20
Male,652,1967.64,3.02
Other / Non-Disclosed,15,50.19,3.35


In [817]:

#merging PC, TPV and APC dataframe with earlier dataframe to cak and add the "Average Total Purchase Per person"
final_pa = pd.merge(merged_pa, gend_dem_df, on="Gender", how="left")
final_pa
atppp = final_pa.iloc[:,2] / final_pa.iloc[:,4]
final_pa["Avg Total Purchase per Person"] = atppp
final_pa['Gender'].astype(str)
final_pa
# reducing to the final "Purchasing Analysis (Gender)" data frame and formmaing
reduced_final_pa = final_pa[["Gender", "Purchase Count", "Average Purchase Price", "Total Purchase Value", 
"Avg Total Purchase per Person"]]
reduced_final_pa.set_index("Gender", inplace = True)
reduced_final_pa.iloc[:, 1] = reduced_final_pa.iloc[:, 1].map("${:.2f}".format)
reduced_final_pa.iloc[:, 2] = reduced_final_pa.iloc[:, 2].map("${:.2f}".format)
reduced_final_pa.iloc[:, 3] = reduced_final_pa.iloc[:, 3].map("${:.2f}".format)
reduced_final_pa


C:\Users\benjy\anaconda3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [818]:
# Create the bins in which Data will be held
# Bins are < 10, 10-14, 15-19, 20-24, 25-29, 30-34, 35-29, 40+.   
# purchase_data_df

# using min and max idenfited 7 as minimn age and 45 as the max
# age_count = purchase_data_df["Age"].min()
# age_count

# creating the bins to hold the age categories from the data
bins = [0, 9, 14, 19, 24, 29, 34, 39, 46]

# Create the names for the  bins
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

# using pd.cut to categorize the existing players uing the age bins
purchase_data_df["Age Bin"] = pd.cut(purchase_data_df["Age"], bins, labels=group_names, include_lowest=True)
# purchase_data_df.info()

#Create a new groupby using the new age bins to get unique count
grp_age = purchase_data_df.groupby(["Age Bin"]).nunique()
grp_age

# Create count of age bins by "SN" columne whichis unique Screen Names
grp_age_reduced = pd.DataFrame(grp_age["SN"])
grp_age_reduced

# Rename "SN" to "Total Count"
grp_age_reduced = grp_age_reduced.rename(
    columns={"SN": "Total Count"})
grp_age_reduced

# create a new column that will calc ulate the % of total players using the previously created "total_players"
#  variable

grp_age_reduced["Percentage of Players"] = round(grp_age_reduced.iloc[:,0] / total_players * 100, 2)
grp_age_reduced["Percentage of Players"] = grp_age_reduced["Percentage of Players"].map("{:.2f}%".format)

# Remove the index name and print the final Age Demographics Table
grp_age_reduced.index.name = None
grp_age_reduced


,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [819]:
# example table included in materials

## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [820]:
# first creating the binned data
grp_age = purchase_data_df.groupby(["Age Bin"]).nunique()
grp_age

# Create a new data frame holding the purchase ID and Rename as "Purchase Count"
age_purchases = pd.DataFrame(grp_age['Purchase ID'])
age_purchases = age_purchases.rename(columns={"Purchase ID": "Purchase Count"})
age_purchases



,Purchase Count
Age Bin,
<10,23
10-14,28
15-19,136
20-24,365
25-29,101
30-34,73
35-39,41
40+,13


In [821]:
# create a new df to hold the total purchases by age bin
grouped_purchase_data_df  = purchase_data_df.groupby(["Age Bin"]).sum()
grouped_purchase_data_df

# Create a new data frame holding the Price and Rename as "total purc hases"
total_purchases = pd.DataFrame(grouped_purchase_data_df['Price'])
total_purchases = total_purchases.rename(columns={"Price": "Total Price"})
total_purchases

,Total Price
Age Bin,
<10,77.13
10-14,82.78
15-19,412.89
20-24,1114.06
25-29,293.00
30-34,214.00
35-39,147.67
40+,38.24


In [822]:
# merging the purchase count and total purchase value
merged_purch_age = pd.merge(age_purchases, total_purchases, on="Age Bin", how="right")
merged_purch_age


# calculated the average purchase price, add to the data frame and map the 
# total purchase value and total purchase prices
average_puchase_price_age = round(merged_purch_age.iloc[:,1] / merged_purch_age.iloc[:,0],2)
merged_purch_age["Average Purchase Price"] = average_puchase_price_age
merged_purch_age

# calculate the average total purchase per person by dividing the total purchase value by the 
# total person count  by age min, rename index, provide cleaner formatting for
# Total Price, Average Purchase Price and Avg Total Price per person and finally print the data frame

merged_purch_age["Avg Total Purchase per Person"] = round(merged_purch_age.iloc[:,1] / grp_age_reduced.iloc[:,0],2)
merged_purch_age.index.names = ["Age Ranges"]
merged_purch_age.iloc[:, 1] = merged_purch_age.iloc[:, 1].map("${:.2f}".format)
merged_purch_age.iloc[:, 2] = merged_purch_age.iloc[:, 2].map("${:.2f}".format)
merged_purch_age.iloc[:, 3] = merged_purch_age.iloc[:, 3].map("${:.2f}".format)
merged_purch_age

,Purchase Count,Total Price,Average Purchase Price,Avg Total Purchase per Person
Age Ranges,,,,
<10,23,$77.13,$3.35,$4.54
10-14,28,$82.78,$2.96,$3.76
15-19,136,$412.89,$3.04,$3.86
20-24,365,$1114.06,$3.05,$4.32
25-29,101,$293.00,$2.90,$3.81
30-34,73,$214.00,$2.93,$4.12
35-39,41,$147.67,$3.60,$4.76
40+,13,$38.24,$2.94,$3.19


In [823]:
# this is the example vew that was provided in the materials.

## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [824]:
# create a new df to hold the total purchases by age bin
grouped_purchase_data_df  = purchase_data_df.groupby(["SN"]).sum()
grouped_purchase_data_df

# Create a new data frame holding the Price and Rename as "total purc hases"
total_purchases_sn = pd.DataFrame(grouped_purchase_data_df['Price'])
total_purchases_sn = total_purchases_sn.rename(columns={"Price": "Total Purchase Value"})
total_purchases_sn.sort_values(["Total Purchase Value"], ascending=False, inplace=True)
total_purchases_sn

,Total Purchase Value
SN,
Lisosia93,18.96
Idastidru52,15.45
Chamjask73,13.83
Iral74,13.62
Iskadarya95,13.10
...,...
Ililsasya43,1.02
Irilis75,1.02
Aidai61,1.01


In [825]:

# Creating Dataframe to hold purchase count by SN
grouped_purchase_sn  = purchase_data_df.groupby(['SN']).count()
grouped_purchase_sn = pd.DataFrame(grouped_purchase_sn['Purchase ID'])
grouped_purchase_sn = grouped_purchase_sn.rename(
    columns={"Purchase ID": "Purchase Count"})
grouped_purchase_sn.sort_values(by=['Purchase Count'])
grouped_purchase_sn




,Purchase Count
SN,
Adairialis76,1
Adastirin33,1
Aeda94,1
Aela59,1
Aelaria33,1
...,...
Yathecal82,3
Yathedeu43,2
Yoishirrala98,1


In [826]:
# merging total purchase value df with purchase count df
merged_ts = pd.merge(grouped_purchase_sn, total_purchases_sn, on="SN", how="right")
merged_ts

,Purchase Count,Total Purchase Value
SN,,
Lisosia93,5,18.96
Idastidru52,4,15.45
Chamjask73,3,13.83
Iral74,4,13.62
Iskadarya95,3,13.10
...,...,...
Ililsasya43,1,1.02
Irilis75,1,1.02
Aidai61,1,1.01


In [827]:
# capture the average purchase price for the newby top spenders
average_puchase_price_ts = round(merged_ts.iloc[:,1] / merged_ts.iloc[:,0],2)

# add a column to hold the average purchase price
merged_ts["Average Purchase Price"] = average_puchase_price_ts

# reorder the columns and format the values
merged_ts = merged_ts[["Purchase Count", "Average Purchase Price", "Total Purchase Value"]]
merged_ts.iloc[:, 1] = merged_ts.iloc[:, 1].map("${:.2f}".format)
merged_ts.iloc[:, 2] = merged_ts.iloc[:, 2].map("${:.2f}".format)

#print just the top five spenders
merged_ts.head(5)

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [8]:
# this is the example materials provided in the materials

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [9]:
# this

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
